### Automated Event Planning

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/crewai

#### Creating Agents

In [4]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",

    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [5]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=ollama_llm,
    allow_delegation=False
)

In [6]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=ollama_llm,
    allow_delegation=False
)

### Creating Venue Pydantic Object
Agents will populate this object with information about different venues by creating different instances of it.

In [7]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [8]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator,
   
)

In [9]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    #async_execution=True,
    agent=logistics_manager,
  
)

In [10]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,

)

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [11]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
           
    
    verbose=True
)

#### Define Input


In [12]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [13]:

result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference venues San Francisco\"}"
## Tool Output: 
{'searchParameters': {'q': 'Tech Innovation Conference venues San Francisco', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'World Agri-Tech Innovation Summit, March 11-12, 2025', 'link': 'https://worldagritechusa.com/', 'snippet': 'The World Agri-Tech Innovation Summit is the stand out event with curated leaders, innovators and investors from across the AgTech landscape.', 'position': 1}, {'title': 'Top 12 tech conferences in San Francisco for 2024 - TravelPerk', 'link': 'https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/', 'snippet': "We've curated our list of the top 12 technology conferences, symposiums, and tech events taking place in San F

 looks good


/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/Users/tharhtet/.local/share/virtualenvs/multiagent-using-multimodal-mViAUUHF/lib/python3.10/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/Use

# Agent: Logistics Manager
## Task: Coordinate catering and equipment for an event with 500 participants on 2024-09-15.


# Agent: Logistics Manager
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"catering options for 500 participants event logistics planning\"}"
## Tool Output: 
{'searchParameters': {'q': 'catering options for 500 participants event logistics planning', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '200 Fresh Event Catering Ideas - Skift Meetings', 'link': 'https://meetings.skift.com/2024/02/12/catering-ideas/', 'snippet': 'This article shares 200 event catering ideas, including ways to make them your own. Some are easy, and others are completely over the top.', 'position': 1}, {'title': 'The Potluck Tales: Lessons in Event Logistics & How to Plan Your ...', 'link': 'https://medium.com/@MarilynsMusings/the-potluck-tales-lessons-in-event-logistics-how-to-plan-your-next-event-well-f1af54f7fec0', 'snippet': "

 looks good


# Agent: Marketing and Communications Agent
## Task: Promote the Tech Innovation Conference aiming to engage at least500 potential attendees.


# Agent: Marketing and Communications Agent
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference marketing strategies for engagement\"}"
## Tool Output: 
{'searchParameters': {'q': 'Tech Innovation Conference marketing strategies for engagement', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '12 Proven Conference Marketing Ideas to Elevate Your Event', 'link': 'https://swoogo.events/blog/conference-marketing-ideas/', 'snippet': 'This guide shares 12 proven marketing strategies to boost attendance, increase engagement, and ensure your event stands out.', 'position': 1}, {'title': '12 Marketing Strategies For Tech Companies with Examples', 'link': 'https://thecmo.com/career-development/marketing-strategies-for-tech-companies/', 'snippet': "From product-led market

In [14]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco',
 'booking_status': '',
 'capacity': 2400,
 'name': 'World Agri-Tech Innovation Summit'}
